In [ ]:
import pandas as pd
from scipy.stats import skew, kurtosis

# Load the dataset
file_path = 'allinone_adjusted_final.xlsx'
data = pd.read_excel(file_path)

# Define your hubs
hubs = ['MidC', 'Palo Verde Peak', 'Indiana', 'SP15 EZ Gen DA LMP Peak']

# Filter data for selected hubs
data_filtered = data[data['Price hub'].isin(hubs)]

# Group by Trade date and hub, and take mean if multiple values exist
grouped = data_filtered.groupby(['Trade date', 'Price hub'])['price'].mean().reset_index()

# Pivot the table to have hubs as columns
pivoted = grouped.pivot(index='Trade date', columns='Price hub', values='price')

# Drop rows with missing values
pivoted.dropna(inplace=True)

# Create summary statistics table
summary = pd.DataFrame(columns=['Mean', 'Standard Deviation', 'Maximum', 'Minimum', 'Skewness', 'Kurtosis'])

for hub in hubs:
    values = pivoted[hub]
    summary.loc[hub] = [
        values.mean(),
        values.std(),
        values.max(),
        values.min(),
        skew(values),
        kurtosis(values, fisher=False)  # use Fisher=False to match Excel-style kurtosis
    ]

# Round values for readability
summary = summary.round(4)

# Display the table
import ace_tools as tools; tools.display_dataframe_to_user(name="Hub Price Statistics", dataframe=summary)
